# 기존 함수 split_morph2

In [ ]:
#최종함수 - include
import pandas as pd
from kiwipiepy import Kiwi
import kss
import re

# Kiwi 초기화
kiwi = Kiwi()

def split_morph2(dataframe, include_tags=None, exclude_tokens=None):
    # 특수 문자 및 숫자 제거를 위한 정규식 패턴
    special_characters = r"[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\xa0\n\d+]"
    # 불용어 리스트 (필요에 따라 추가 가능)
    stopwords = ["그리고", "그래서", "하지만"]

    def clean_text(text):
        # 특수 문자 제거
        text = re.sub(special_characters, "", text)
        # 불용어 제거
        words = text.split()
        words = [word for word in words if word not in stopwords]
        return " ".join(words)
    
    rc = dataframe['review_content']  # df에서 review content 추출
    crc = dataframe['corrected_review_content']  # df에서 corrected review content 추출
    
    # kss로 문장 단위로 나누기 - rc
    rc_split = []
    for content in rc:
        try:
            sentences = kss.split_sentences(content)  # 문장 단위로 분리
            rc_split.append(sentences)
        except Exception as e:
            rc_split.append([])  # 오류 발생 시 빈 리스트 추가
    
    # kss로 문장 단위로 나누기 - crc
    crc_split = []
    for content in crc:
        try:
            sentences = kss.split_sentences(content)  # 문장 단위로 분리
            crc_split.append(sentences)
        except Exception as e:
            crc_split.append([])  # 오류 발생 시 빈 리스트 추가
    
    # 각 문장에 대해 형태소 분석을 수행하고 그 결과를 리스트에 저장 - rc
    rc_morphs = []
    for sentences in rc_split:
        sentence_morphs = []
        for sentence in sentences:
            cleaned_sentence = clean_text(sentence)  # 불용어 및 특수 문자 제거
            morphs = kiwi.analyze(cleaned_sentence)[0][0]  # 형태소와 태그 추출
            if include_tags:
                morphs = [token.form for token in morphs if token.tag in include_tags]  # include_tags에 해당하는 형태소만 추출
            if exclude_tokens:
                # 제거하고 싶은 형태소를 제외
                morphs = [token for token in morphs if token not in exclude_tokens]
            
            sentence_morphs.append(' '.join(morphs))
        rc_morphs.append(' '.join(sentence_morphs))
    
    rc_morph_final = pd.Series(rc_morphs)
    
    # 각 문장에 대해 형태소 분석을 수행하고 그 결과를 리스트에 저장 - crc
    crc_morphs = []
    for sentences in crc_split:
        sentence_morphs = []
        for sentence in sentences:
            cleaned_sentence = clean_text(sentence)  # 불용어 및 특수 문자 제거
            morphs = kiwi.analyze(cleaned_sentence)[0][0]  # 형태소와 태그 추출
            if include_tags:
                morphs = [token.form for token in morphs if token.tag in include_tags]  # include_tags에 해당하는 형태소만 추출
            if exclude_tokens:
            # 제거하고 싶은 형태소를 제외
                morphs = [token for token in morphs if token not in exclude_tokens]
            sentence_morphs.append(' '.join(morphs))
        crc_morphs.append(' '.join(sentence_morphs))
    
    crc_morph_final = pd.Series(crc_morphs)
    
    return rc_morph_final, crc_morph_final

# 수정 함수 - split_morph3   
=> 적용

In [4]:
#최종함수 - include
import pandas as pd
from kiwipiepy import Kiwi
import kss
import re

# Kiwi 초기화
kiwi = Kiwi()

def split_morph3(dataframe, include_tags=None, exclude_tokens=None):
    # 특수 문자 및 숫자 제거를 위한 정규식 패턴
    special_characters = r"[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\xa0\n\d+]"
    # 불용어 리스트 (필요에 따라 추가 가능)
    stopwords = ["그리고", "그래서", "하지만"]

    def clean_text(text):
        # 특수 문자 제거
        text = re.sub(special_characters, "", text)
        # 불용어 제거
        words = text.split()
        words = [word for word in words if word not in stopwords]
        return " ".join(words)
    
    rc = dataframe['review_content']  # df에서 review content 추출
    crc = dataframe['corrected_review_content']  # df에서 corrected review content 추출
    
    # kss로 문장 단위로 나누기 - rc - 리스트 형태로 반환하여 추가 수정 필요
    rc_split = []
    for content in rc:
        try:
            sentences = kss.split_sentences(content)  # 문장 단위로 분리
            rc_split.extend(sentences)
        except Exception as e:
            pass  # 오류 발생 시 pass
    
    
    # kss로 문장 단위로 나누기 - crc
    crc_split = []
    for content in crc:
        try:
            sentences = kss.split_sentences(content)  # 문장 단위로 분리
            crc_split.extend(sentences)
        except Exception as e:
            pass  # 오류 발생 시 pass
    
    # 각 문장에 대해 형태소 분석을 수행하고 그 결과를 리스트에 저장 - rc
    rc_morphs = []
    for sentence in rc_split:
        cleaned_sentence = clean_text(sentence)  # 불용어 및 특수 문자 제거
        morphs = kiwi.analyze(cleaned_sentence)[0][0]  # 형태소와 태그 추출
        if include_tags:
            morphs = [token.form for token in morphs if token.tag in include_tags]  # include_tags에 해당하는 형태소만 추출
        if exclude_tokens:
            # 제거하고 싶은 형태소를 제외
            morphs = [token for token in morphs if token not in exclude_tokens]
        
        token_morphs = ' '.join(morphs)        
        rc_morphs.append(token_morphs) # 리스트 단위로 추가
    
    rc_morph_series = pd.Series(rc_morphs)
    rc_morph_final = rc_morph_series.dropna()
    
    # 각 문장에 대해 형태소 분석을 수행하고 그 결과를 리스트에 저장 - crc
    crc_morphs = []
    for sentence in crc_split:
        cleaned_sentence = clean_text(sentence)  # 불용어 및 특수 문자 제거
        morphs = kiwi.analyze(cleaned_sentence)[0][0]  # 형태소와 태그 추출
        if include_tags:
            morphs = [token.form for token in morphs if token.tag in include_tags]  # include_tags에 해당하는 형태소만 추출
        if exclude_tokens:
            # 제거하고 싶은 형태소를 제외
            morphs = [token for token in morphs if token not in exclude_tokens]
        
        token_morphs = ' '.join(morphs)
        crc_morphs.append(token_morphs)
    
    crc_morph_series = pd.Series(crc_morphs)
    crc_morph_final = crc_morph_series.dropna()
    
    return rc_morph_final, crc_morph_final

In [3]:
# top, all 파일 불러오기

import pandas as pd

skincare_top = pd.read_csv('/Users/junsungrhee/pyenv_3.10/files/6. final project/skincare_top30맞춤법검사.csv')
skincare_all = pd.read_csv('/Users/junsungrhee/pyenv_3.10/files/6. final project/skincare_all맞춤법검사.csv')
suncare_top = pd.read_csv('/Users/junsungrhee/pyenv_3.10/files/6. final project/suncareTOP30맞춤법.csv')
suncare_all = pd.read_csv('/Users/junsungrhee/pyenv_3.10/files/6. final project/suncare_all맞춤법검사.csv')

### 전처리 함수 적용 최종

In [11]:
# skincare_top 전처리2 : '좋' 제거안한 경우

include_tag = [
    'NNG',  # 일반 명사 (Common Noun)
    'NNP',  # 고유 명사 (Proper Noun)
    'NNB',  # 의존 명사 (Bound Noun)
    'NR',   # 수사 (Numeral)
    'NP',   # 대명사 (Pronoun)
    'VV',   # 동사 (Verb)
    'VX',   # 보조 동사 (Auxiliary Verb)
    'VA',   # 형용사 (Adjective)
    'MAG',  # 일반 부사 (General Adverb)
    'MAJ',  # 접속 부사 (Conjunctive Adverb)
    'XR',   # 어근 (Root)
    'SL',   # 외국어 (Foreign Language)
    'SH',   # 한자 (Chinese Character)
    'EP',   # 선어말 어미 (Pre-final Ending)
]

exclude_token = ['하','었', '지', '겠', '거','것']

skincare_top_rc2, skincare_top_crc2 = split_morph3(skincare_top, include_tags=include_tag,exclude_tokens=exclude_token)

skincare_top_rc2.to_excel('/Users/junsungrhee/pyenv_3.10/files/6. final project/skincare,suncare 전처리, 문장나누기/skincare_top_rc2.xlsx', index=False)
skincare_top_crc2.to_excel('/Users/junsungrhee/pyenv_3.10/files/6. final project/skincare,suncare 전처리, 문장나누기/skincare_top_crc2.xlsx', index=False)

In [12]:
# skincare_top 전처리3 : '좋' 제거한 경우

include_tag = [
    'NNG',  # 일반 명사 (Common Noun)
    'NNP',  # 고유 명사 (Proper Noun)
    'NNB',  # 의존 명사 (Bound Noun)
    'NR',   # 수사 (Numeral)
    'NP',   # 대명사 (Pronoun)
    'VV',   # 동사 (Verb)
    'VX',   # 보조 동사 (Auxiliary Verb)
    'VA',   # 형용사 (Adjective)
    'MAG',  # 일반 부사 (General Adverb)
    'MAJ',  # 접속 부사 (Conjunctive Adverb)
    'XR',   # 어근 (Root)
    'SL',   # 외국어 (Foreign Language)
    'SH',   # 한자 (Chinese Character)
    'EP',   # 선어말 어미 (Pre-final Ending)
]

exclude_token = ['하','었', '지', '좋', '겠', '거','것']

skincare_top_rc3, skincare_top_crc3 = split_morph3(skincare_top, include_tags=include_tag,exclude_tokens=exclude_token)

skincare_top_rc3.to_excel('/Users/junsungrhee/pyenv_3.10/files/6. final project/skincare,suncare 전처리, 문장나누기/skincare_top_rc3.xlsx', index=False)
skincare_top_crc3.to_excel('/Users/junsungrhee/pyenv_3.10/files/6. final project/skincare,suncare 전처리, 문장나누기/skincare_top_crc3.xlsx', index=False)

In [13]:
# suncare_top 전처리2 : '좋' 제거안한 경우

include_tag = [
    'NNG',  # 일반 명사 (Common Noun)
    'NNP',  # 고유 명사 (Proper Noun)
    'NNB',  # 의존 명사 (Bound Noun)
    'NR',   # 수사 (Numeral)
    'NP',   # 대명사 (Pronoun)
    'VV',   # 동사 (Verb)
    'VX',   # 보조 동사 (Auxiliary Verb)
    'VA',   # 형용사 (Adjective)
    'MAG',  # 일반 부사 (General Adverb)
    'MAJ',  # 접속 부사 (Conjunctive Adverb)
    'XR',   # 어근 (Root)
    'SL',   # 외국어 (Foreign Language)
    'SH',   # 한자 (Chinese Character)
    'EP',   # 선어말 어미 (Pre-final Ending)
]

exclude_token = ['하','었', '지', '겠', '거','것']

suncare_top_rc2, suncare_top_crc2 = split_morph3(suncare_top, include_tags=include_tag,exclude_tokens=exclude_token)

suncare_top_rc2.to_excel('/Users/junsungrhee/pyenv_3.10/files/6. final project/skincare,suncare 전처리, 문장나누기/suncare_top_rc2.xlsx', index=False)
suncare_top_crc2.to_excel('/Users/junsungrhee/pyenv_3.10/files/6. final project/skincare,suncare 전처리, 문장나누기/suncare_top_crc2.xlsx', index=False)

In [14]:
# suncare_top 전처리3 : '좋' 제거한 경우

include_tag = [
    'NNG',  # 일반 명사 (Common Noun)
    'NNP',  # 고유 명사 (Proper Noun)
    'NNB',  # 의존 명사 (Bound Noun)
    'NR',   # 수사 (Numeral)
    'NP',   # 대명사 (Pronoun)
    'VV',   # 동사 (Verb)
    'VX',   # 보조 동사 (Auxiliary Verb)
    'VA',   # 형용사 (Adjective)
    'MAG',  # 일반 부사 (General Adverb)
    'MAJ',  # 접속 부사 (Conjunctive Adverb)
    'XR',   # 어근 (Root)
    'SL',   # 외국어 (Foreign Language)
    'SH',   # 한자 (Chinese Character)
    'EP',   # 선어말 어미 (Pre-final Ending)
]

exclude_token = ['하','었', '지', '좋', '겠', '거','것']

suncare_top_rc3, suncare_top_crc3 = split_morph3(suncare_top, include_tags=include_tag,exclude_tokens=exclude_token)

suncare_top_rc3.to_excel('/Users/junsungrhee/pyenv_3.10/files/6. final project/skincare,suncare 전처리, 문장나누기/suncare_top_rc3.xlsx', index=False)
suncare_top_crc3.to_excel('/Users/junsungrhee/pyenv_3.10/files/6. final project/skincare,suncare 전처리, 문장나누기/suncare_top_crc3.xlsx', index=False)

In [6]:
#skincare all 전처리 : : '좋' 제거안함
#include_tag, exclude_token

import os

include_tag = [
    'NNG',  # 일반 명사 (Common Noun)
    'NNP',  # 고유 명사 (Proper Noun)
    'NNB',  # 의존 명사 (Bound Noun)
    'NR',   # 수사 (Numeral)
    'NP',   # 대명사 (Pronoun)
    'VV',   # 동사 (Verb)
    'VX',   # 보조 동사 (Auxiliary Verb)
    'VA',   # 형용사 (Adjective)
    'MAG',  # 일반 부사 (General Adverb)
    'MAJ',  # 접속 부사 (Conjunctive Adverb)
    'XR',   # 어근 (Root)
    'SL',   # 외국어 (Foreign Language)
    'SH',   # 한자 (Chinese Character)
    'EP',   # 선어말 어미 (Pre-final Ending)
    'EF'    # 종결 어미 (Final Ending)
]

exclude_token = ['하','었', '지','좋', '겠', '거','것']

skincare_dfs = {}
skincare_rc = {}
skincare_crc = {}

for i in range(1,39+1):
    skincare_dfs = skincare_all[skincare_all['제품id']== f'skincare{i}']   
    skincare_rc, skincare_crc = split_morph3(skincare_dfs, include_tags=include_tag, exclude_tokens=exclude_token)
    # 생성된 데이터프레임을 각각 CSV 파일로 저장
    rc_filename = os.path.join('/Users/junsungrhee/pyenv_3.10/files/6. final project/skincare,suncare 전처리, 문장나누기/제품', f'skincare{i}_rc.xlsx')
    crc_filename = os.path.join('/Users/junsungrhee/pyenv_3.10/files/6. final project/skincare,suncare 전처리, 문장나누기/제품', f'skincare{i}_crc.xlsx')
    
    skincare_rc.to_excel(rc_filename, index=False)
    skincare_crc.to_excel(crc_filename, index=False)
    

In [5]:
#suncare all 전처리: : '좋' 제거안함
#include_tag, exclude_token

import os

include_tag = [
    'NNG',  # 일반 명사 (Common Noun)
    'NNP',  # 고유 명사 (Proper Noun)
    'NNB',  # 의존 명사 (Bound Noun)
    'NR',   # 수사 (Numeral)
    'NP',   # 대명사 (Pronoun)
    'VV',   # 동사 (Verb)
    'VX',   # 보조 동사 (Auxiliary Verb)
    'VA',   # 형용사 (Adjective)
    'MAG',  # 일반 부사 (General Adverb)
    'MAJ',  # 접속 부사 (Conjunctive Adverb)
    'XR',   # 어근 (Root)
    'SL',   # 외국어 (Foreign Language)
    'SH',   # 한자 (Chinese Character)
    'EP',   # 선어말 어미 (Pre-final Ending)
    'EF'    # 종결 어미 (Final Ending)
]

exclude_token = ['하','었', '지', '겠', '좋', '거','것']

suncare_dfs = {}
suncare_rc = {}
suncare_crc = {}

for i in range(1,9+1):
    suncare_dfs = suncare_all[suncare_all['제품id']== f'suncare{i}']   
    suncare_rc, suncare_crc = split_morph3(suncare_dfs, include_tags=include_tag, exclude_tokens=exclude_token)
    # 생성된 데이터프레임을 각각 CSV 파일로 저장
    rc_filename = os.path.join('/Users/junsungrhee/pyenv_3.10/files/6. final project/skincare,suncare 전처리, 문장나누기/제품', f'suncare{i}_rc.xlsx')
    crc_filename = os.path.join('/Users/junsungrhee/pyenv_3.10/files/6. final project/skincare,suncare 전처리, 문장나누기/제품', f'suncare{i}_crc.xlsx')
    
    suncare_rc.to_excel(rc_filename, index=False)
    suncare_crc.to_excel(crc_filename, index=False)
    

[Kss]: Oh! You have konlpy.tag.Mecab in your environment. Kss will take this as a backend! :D

